In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer, Wav2Vec2Model, Wav2Vec2Processor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import torchaudio

# Paths to audio and text data
AUDIO_FOLDER = "./malayalam/audio/"
TEXT_FILE = "./malayalam/text/ML-AT-train.csv"

# Hyperparameters
BATCH_SIZE = 1
HIDDEN_SIZE = 512
NUM_LSTM_LAYERS = 2
DROPOUT_PROB = 0.3
LEARNING_RATE = 2e-5
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torchaudio.transforms as transforms
from transformers import AutoTokenizer

class MultiModalDataset(Dataset):
    def __init__(self, audio_paths, transcripts, labels, audio_processor, text_tokenizer, max_length=48):
        self.audio_paths = audio_paths
        self.transcripts = transcripts
        self.labels = labels
        self.audio_processor = audio_processor
        self.text_tokenizer =  text_tokenizer
        self.max_length = max_length
        self.resampler = transforms.Resample(orig_freq=44100, new_freq=16000)  # Adjust orig_freq as needed

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load audio file
        audio_path = self.audio_paths[idx]
        waveform, sr = torchaudio.load(audio_path)

        # Resample audio if necessary
        if sr != 16000:
            waveform = self.resampler(waveform)

        # Preprocess audio
        audio_features = self.audio_processor(
            waveform.squeeze().numpy(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        ).input_values[0]

        # Tokenize transcript
        transcript = self.transcripts[idx]
        text_encoding = self.text_tokenizer(
            transcript,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Extract input_ids and attention_mask
        input_ids = text_encoding['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = text_encoding['attention_mask'].squeeze(0)  # Remove batch dimension

        # Label
        label = self.labels[idx]

        return {
            "audio_features": audio_features,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": torch.tensor(label, dtype=torch.long)
        }


class BilstmXLMRobertaWav2VecClassifier(nn.Module):
    def __init__(self, xlm_model_name: str, wav2vec_model_name: str, num_labels: int,
                 lstm_hidden_size: int, lstm_layers: int, dropout_prob: float):
        super(BilstmXLMRobertaWav2VecClassifier, self).__init__()

        # Text embeddings using XLM-RoBERTa
        self.roberta = AutoModel.from_pretrained(
            xlm_model_name, cache_dir="./telugu_lm"
        )

        # Speech embeddings using Wav2Vec 2.0
        self.wav2vec = Wav2Vec2Model.from_pretrained(
            wav2vec_model_name
        )

        # BiLSTM for combined embeddings
        self.bilstm = nn.LSTM(
            input_size=768,  # Combined hidden sizes
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            bidirectional=True,
            batch_first=True
        )

        # Attention mechanism
        self.attention = nn.Linear(lstm_hidden_size * 2, 1)

        # Dropout and layer normalization
        self.dropout = nn.Dropout(dropout_prob)
        self.layer_norm = nn.LayerNorm(lstm_hidden_size * 2)

        # Final classification layer
        self.classifier = nn.Linear(lstm_hidden_size * 2, num_labels)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor,
                audio_features: torch.Tensor, labels: torch.Tensor = None):
        # Text embeddings from XLM-RoBERTa
        text_outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        text_embeddings = text_outputs.last_hidden_state  # (batch_size, seq_len, roberta_hidden_size)

        # Speech embeddings from Wav2Vec
        speech_outputs = self.wav2vec(audio_features)
        speech_embeddings = speech_outputs.last_hidden_state  # (batch_size, seq_len, wav2vec_hidden_size)
        
        # Concatenate text and speech embeddings
        combined_embeddings = torch.cat([text_embeddings, speech_embeddings], dim=1)  # (batch_size, total_seq_len, combined_hidden_size)

        # BiLSTM for combined embeddings
        lstm_output, _ = self.bilstm(combined_embeddings)  # (batch_size, total_seq_len, lstm_hidden_size * 2)

        # Attention mechanism
        attention_weights = torch.softmax(torch.tanh(self.attention(lstm_output)), dim=1)  # (batch_size, total_seq_len, 1)
        attention_output = torch.sum(lstm_output * attention_weights, dim=1)  # (batch_size, lstm_hidden_size * 2)

        # Normalize and apply dropout
        attention_output = self.layer_norm(attention_output)
        attention_output = self.dropout(attention_output)

        # Classification logits
        logits = self.classifier(attention_output)  # (batch_size, num_labels)

        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return (loss, logits) if loss is not None else logits


# Load dataset
def load_data(audio_folder, text_file):
    audio_paths, transcripts, labels = [], [], []
    df = pd.read_csv(text_file)
    for _, row in df.iterrows():
        file_name = row["File Name"]
        transcript = row["Transcript"]
        label = row["Class Label Short"]

        audio_path = os.path.join(audio_folder, file_name + ".wav")
        if os.path.exists(audio_path):
            audio_paths.append(audio_path)
            transcripts.append(transcript)
            labels.append(label)
        else:
            print(f"Audio file not found: {audio_path}")

    return audio_paths, transcripts, labels


# Model names
xlm_model_name = "l3cube-pune/malayalam-topic-all-doc"
wav2vec_model_name = "./ml_w2v"

# Load processors
audio_processor = Wav2Vec2Processor.from_pretrained(wav2vec_model_name)
text_tokenizer = AutoTokenizer.from_pretrained(xlm_model_name, cache_dir="./malayalam_lm")

# Load data
train_audio, train_transcripts, train_labels = load_data(AUDIO_FOLDER, TEXT_FILE)
unique_labels = list(set(train_labels))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Labels:", label_mapping)
NUM_CLASSES = len(unique_labels)

train_labels = [label_mapping[label] for label in train_labels]

test_text_file = "./test/malayalam/text/ML-AT-test.csv"
test_audio_folder = "./test/malayalam/audio/"
test_label_paths = "ML-AT-test.xlsx - Sheet1.csv"
df_test1 = pd.read_csv(test_label_paths)
print(list(df_test1))
test_pathsMap = {path:label for path, label in zip(df_test1["File Name"],df_test1["Class Label"])}

val_audio, val_transcripts, val_labels = [], [], []
df_test = pd.read_csv(test_text_file)
for name, trans in zip(df_test["File Name"],df_test["Transcript"]):
    label = test_pathsMap[name]
    val_labels.append(label_mapping[label])
    audio_path = os.path.join(test_audio_folder, name + ".wav")
    val_audio.append(audio_path)
    val_transcripts.append(trans)

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    audio_features = [item['audio_features'] for item in batch]
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)

    # Pad sequences
    audio_features_padded = pad_sequence(audio_features, batch_first=True)
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {
        'audio_features': audio_features_padded,
        'input_ids': input_ids_padded,
        'attention_mask': attention_masks_padded,
        'label': labels
    }



# Create datasets and dataloaders
train_dataset = MultiModalDataset(train_audio, train_transcripts, train_labels, audio_processor, text_tokenizer)
val_dataset = MultiModalDataset(val_audio, val_transcripts, val_labels, audio_processor, text_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Initialize the model
model = BilstmXLMRobertaWav2VecClassifier(
    xlm_model_name=xlm_model_name,
    wav2vec_model_name=wav2vec_model_name,
    num_labels=NUM_CLASSES,
    lstm_hidden_size=HIDDEN_SIZE,
    lstm_layers=NUM_LSTM_LAYERS,
    dropout_prob=DROPOUT_PROB
).to(DEVICE)


# Optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Initialize variables for saving the best model
best_macro_f1 = 0.0
best_model_path = "best_model_ml1.pth"

# Training and validation loop
for epoch in range(NUM_EPOCHS):
    # Training
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} [Training]"):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        audio_features = batch["audio_features"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, audio_features=audio_features, labels=labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} [Validation]"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            audio_features = batch["audio_features"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            loss, logits = model(input_ids=input_ids, attention_mask=attention_mask, audio_features=audio_features, labels=labels)
            total_val_loss += loss.item()

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    report = classification_report(all_labels, all_preds)
    print(report)
    
    # Generate the classification report as a dictionary
    report1 = classification_report(all_labels,all_preds,output_dict=True)
    # Extract macro average F1-score
    macro_f1 = report1['macro avg']['f1-score']

    # Save the model if it has the best macro F1-score
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(model.state_dict(), best_model_path)
        print(f"New best Macro F1-Score: {best_macro_f1:.4f}. Saving model...")
    else:
        print(f"Macro Average F1-Score: {macro_f1:.4f}")

    # Adjust learning rate based on validation loss
    scheduler.step(avg_val_loss)

    



Labels: {'C': 0, 'G': 1, 'N': 2, 'P': 3, 'R': 4}
['File Name', 'Class Label']


Epoch 1/10 [Training]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 883/883 [04:08<00:00,  3.56it/s]


Epoch 1, Training Loss: 1.3087


Epoch 1/10 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 14.07it/s]
/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/scikit_learn-1.5.1-py3.11-linux-x86_64.egg/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/scikit_learn-1.5.1-py3.11-linux-x86_64.egg/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"

Epoch 1, Validation Loss: 1.0899, Accuracy: 0.5600

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       0.00      0.00      0.00        10
           2       0.83      1.00      0.91        10
           3       0.33      0.90      0.49        10
           4       0.00      0.00      0.00        10

    accuracy                           0.56        50
   macro avg       0.40      0.56      0.45        50
weighted avg       0.40      0.56      0.45        50

New best Macro F1-Score: 0.4505. Saving model...


...

Epoch 9, Validation Loss: 1.5748, Accuracy: 0.8400

Classification Report:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        10
           1       1.00      0.70      0.82        10
           2       0.83      1.00      0.91        10
           3       0.88      0.70      0.78        10
           4       0.80      0.80      0.80        10

    accuracy                           0.84        50
   macro avg       0.86      0.84      0.84        50
weighted avg       0.86      0.84      0.84        50

Macro Average F1-Score: 0.8360


Epoch 10/10 [Training]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 883/883 [04:02<00:00,  3.65it/s]


Epoch 10, Training Loss: 0.1251


Epoch 10/10 [Validation]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 12.44it/s]

Epoch 10, Validation Loss: 1.5910, Accuracy: 0.8400

Classification Report:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        10
           1       1.00      0.70      0.82        10
           2       0.83      1.00      0.91        10
           3       0.88      0.70      0.78        10
           4       0.80      0.80      0.80        10

    accuracy                           0.84        50
   macro avg       0.86      0.84      0.84        50
weighted avg       0.86      0.84      0.84        50

Macro Average F1-Score: 0.8360


In [3]:
import pandas as pd
from tqdm import tqdm
import torch
import torchaudio
import torchaudio.transforms as transforms
from torch.utils.data import Dataset


class SingleSampleMultiModalDataset(Dataset):
    def __init__(self, audio_path, transcript, audio_processor, text_tokenizer, max_length=48):
        self.audio_path = audio_path
        self.transcript = transcript
        self.audio_processor = audio_processor
        self.text_tokenizer = text_tokenizer
        self.max_length = max_length
        self.resampler = transforms.Resample(orig_freq=44100, new_freq=16000)

    def __len__(self):
        return 1

    def __getitem__(self, idx=None):
        waveform, sr = torchaudio.load(self.audio_path)
        if sr != 16000:
            waveform = self.resampler(waveform)

        audio_features = self.audio_processor(
            waveform.squeeze().numpy(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        ).input_values[0]

        text_encoding = self.text_tokenizer(
            self.transcript,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = text_encoding['input_ids'].squeeze(0)
        attention_mask = text_encoding['attention_mask'].squeeze(0)

        return {
            "audio_features": audio_features,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        }


# Load the best model
model = BilstmXLMRobertaWav2VecClassifier(
    xlm_model_name=xlm_model_name,
    wav2vec_model_name=wav2vec_model_name,
    num_labels=NUM_CLASSES,
    lstm_hidden_size=HIDDEN_SIZE,
    lstm_layers=NUM_LSTM_LAYERS,
    dropout_prob=DROPOUT_PROB
).to(DEVICE)
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
model.eval()

# Reverse label mapping
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Load test data
test_text_file = "./test/malayalam/text/ML-AT-test.csv"
test_audio_folder = "./test/malayalam/audio/"
df_test = pd.read_csv(test_text_file)

# Inference loop
predictions = []
for idx, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Inference"):
    file_name = row["File Name"]
    transcript = row["Transcript"]
    audio_path = f"{test_audio_folder}/{file_name}.wav"

    # Create dataset instance for the sample
    dataset = SingleSampleMultiModalDataset(audio_path, transcript, audio_processor, text_tokenizer)
    processed_data = dataset[0]  # Access the single sample

    # Move tensors to device
    input_ids = processed_data["input_ids"].unsqueeze(0).to(DEVICE)  # Add batch dimension
    attention_mask = processed_data["attention_mask"].unsqueeze(0).to(DEVICE)
    audio_features = processed_data["audio_features"].unsqueeze(0).to(DEVICE)

    # Model prediction
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask, audio_features=audio_features)
        pred_label = torch.argmax(logits, dim=1).item()

    # Append results
    predictions.append([file_name, reverse_label_mapping[pred_label]])

# Save predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=["File Name", "Class Label Short"])

# Save predictions to a TSV file
output_csv = "byteSizedLLM_malayalam_run1.tsv"
predictions_df.to_csv(output_csv, sep="\t", index=False)
print(f"Inference complete. Predictions saved to '{output_csv}'.")



/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly

Inference complete. Predictions saved to 'byteSizedLLM_malayalam_run1.tsv'.


In [5]:
import pandas as pd
from tqdm import tqdm
import torch
import torchaudio
import torchaudio.transforms as transforms
from torch.utils.data import Dataset


class SingleSampleMultiModalDataset(Dataset):
    def __init__(self, audio_path, transcript, audio_processor, text_tokenizer, max_length=48):
        self.audio_path = audio_path
        self.transcript = transcript
        self.audio_processor = audio_processor
        self.text_tokenizer = text_tokenizer
        self.max_length = max_length
        self.resampler = transforms.Resample(orig_freq=44100, new_freq=16000)

    def __len__(self):
        return 1

    def __getitem__(self, idx=None):
        waveform, sr = torchaudio.load(self.audio_path)
        if sr != 16000:
            waveform = self.resampler(waveform)

        audio_features = self.audio_processor(
            waveform.squeeze().numpy(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        ).input_values[0]

        text_encoding = self.text_tokenizer(
            self.transcript,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = text_encoding['input_ids'].squeeze(0)
        attention_mask = text_encoding['attention_mask'].squeeze(0)

        return {
            "audio_features": audio_features,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        }


model.eval()

# Reverse label mapping
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Load test data
test_text_file = "./test/malayalam/text/ML-AT-test.csv"
test_audio_folder = "./test/malayalam/audio/"
df_test = pd.read_csv(test_text_file)

# Inference loop
predictions = []
for idx, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Inference"):
    file_name = row["File Name"]
    transcript = row["Transcript"]
    audio_path = f"{test_audio_folder}/{file_name}.wav"

    # Create dataset instance for the sample
    dataset = SingleSampleMultiModalDataset(audio_path, transcript, audio_processor, text_tokenizer)
    processed_data = dataset[0]  # Access the single sample

    # Move tensors to device
    input_ids = processed_data["input_ids"].unsqueeze(0).to(DEVICE)  # Add batch dimension
    attention_mask = processed_data["attention_mask"].unsqueeze(0).to(DEVICE)
    audio_features = processed_data["audio_features"].unsqueeze(0).to(DEVICE)

    # Model prediction
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask, audio_features=audio_features)
        pred_label = torch.argmax(logits, dim=1).item()

    # Append results
    predictions.append([file_name, reverse_label_mapping[pred_label]])

# Save predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=["File Name", "Class Label Short"])

# Save predictions to a TSV file
output_csv = "byteSizedLLM_malayalam_run2.tsv"
predictions_df.to_csv(output_csv, sep="\t", index=False)
print(f"Inference complete. Predictions saved to '{output_csv}'.")



Inference: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 12.49it/s]

Inference complete. Predictions saved to 'byteSizedLLM_malayalam_run2.tsv'.
